## Embeddings to check

* Word Embeddings (word2vec, GloVe, etc.)
* Starspace and other similarity learning embeddings
* char-gram embeddings (bpemb, fasttext etc.)
* doc and sentence embeddings (doc2vec, sent2vec etc.)
* specific context models (BERT, ELMo)
* Advanced: Poincare Embeddings concept

# Libs import

In [0]:
import os, math, operator, csv, random, pickle, re, sys
# import tensorflow as tf
from tqdm import tqdm
import pandas as pd
import gc
from collections import Counter

from nltk.util import pad_sequence

import numpy as np
import matplotlib.pyplot as plt

import gc
# from keras import backend as K

import nltk
nltk.download('stopwords');
from nltk.tokenize import TweetTokenizer

from nltk.corpus import stopwords

! pip install Unidecode;
from unidecode import unidecode
#! pip install pyspellchecker;
#from spellchecker import SpellChecker

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import KFold, train_test_split, cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB, MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score, accuracy_score

import numpy as np
import pandas as pd

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
     |████████████████████████████████| 245kB 2.8MB/s 


# Global vars

In [0]:
COLAB = True

# Authorization on Google drive and configurings paths

In [0]:
if COLAB:
    from google.colab import drive
    drive.mount('/content/drive')

    data_folder = '/content/drive/My Drive/Advanced NLP/Homework 1: Classical classification task like Kaggle Toxic or Quora/Toxic data'
    embeddings_folder = '/content/drive/My Drive/Advanced NLP/Homework 1: Classical classification task like Kaggle Toxic or Quora'
    print('data found:', os.listdir(data_folder))
else:
    data_folder = '../input/jigsaw-toxic-comment-classification-challenge/'
    embeddings_folder = '../input/glove-global-vectors-for-word-representation'
    print('data found:', os.listdir(data_folder))

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive
data found: ['test.csv', 'test_labels.csv', 'train.csv', 'sample_submission.csv', 'submission.csv']


In [0]:
#pretrained_folder = "../input/"
train_filepath = os.path.join(data_folder,"train.csv")
test_filepath = os.path.join(data_folder,"test.csv")
test_labels_filepath = os.path.join(data_folder,"test_labels.csv")

#path to a submission
submission_path = os.path.join(data_folder,"sample_submission.csv")

# Preprocessing

## Dicts and lists of useful words and transformations

In [0]:
#List of some words that often appear in toxic comments
#Sorry about the level of toxicity in it!
toxic_words = ["poop", "crap", "prick", "twat", "wikipedia", "wiki", "hahahahaha", "lol", "bastard", "sluts", "slut", "douchebag", "douche", "blowjob", "nigga", "dumb", "jerk", "wanker", "wank", "penis", "motherfucker", "fucker", "fuk", "fucking", "fucked", "fuck", "bullshit", "shit", "stupid", "bitches", "bitch", "suck", "cunt", "dick", "cocks", "cock", "die", "kill", "gay", "jewish", "jews", "jew", "niggers", "nigger", "faggot", "fag", "asshole"]
astericks_words = [('mother****ers', 'motherfuckers'), ('motherf*cking', 'motherfucking'), ('mother****er', 'motherfucker'), ('motherf*cker', 'motherfucker'), ('bullsh*t', 'bullshit'), ('f**cking', 'fucking'), ('f*ucking', 'fucking'), ('fu*cking', 'fucking'), ('****ing', 'fucking'), ('a**hole', 'asshole'), ('assh*le', 'asshole'), ('f******', 'fucking'), ('f*****g', 'fucking'), ('f***ing', 'fucking'), ('f**king', 'fucking'), ('f*cking', 'fucking'), ('fu**ing', 'fucking'), ('fu*king', 'fucking'), ('fuc*ers', 'fuckers'), ('f*****', 'fucking'), ('f***ed', 'fucked'), ('f**ker', 'fucker'), ('f*cked', 'fucked'), ('f*cker', 'fucker'), ('f*ckin', 'fucking'), ('fu*ker', 'fucker'), ('fuc**n', 'fucking'), ('ni**as', 'niggas'), ('b**ch', 'bitch'), ('b*tch', 'bitch'), ('c*unt', 'cunt'), ('f**ks', 'fucks'), ('f*ing', 'fucking'), ('ni**a', 'nigga'), ('c*ck', 'cock'), ('c*nt', 'cunt'), ('cr*p', 'crap'), ('d*ck', 'dick'), ('f***', 'fuck'), ('f**k', 'fuck'), ('f*ck', 'fuck'), ('fc*k', 'fuck'), ('fu**', 'fuck'), ('fu*k', 'fuck'), ('s***', 'shit'), ('s**t', 'shit'), ('sh**', 'shit'), ('sh*t', 'shit'), ('tw*t', 'twat')]
fasttext_misspelings = {"'n'balls": 'balls', "-nazi's": 'nazis', 'adminabuse': 'admin abuse', "admins's": 'admins', 'arsewipe': 'arse wipe', 'assfack': 'asshole', 'assholifity': 'asshole', 'assholivity': 'asshole', 'asshoul': 'asshole', 'asssholeee': 'asshole', 'belizeans': 'mexicans', "blowing's": 'blowing', 'bolivians': 'mexicans', 'celtofascists': 'fascists', 'censorshipmeisters': 'censor', 'chileans': 'mexicans', 'clerofascist': 'fascist', 'cowcrap': 'crap', 'crapity': 'crap', "d'idiots": 'idiots', 'deminazi': 'nazi', 'dftt': "don't feed the troll", 'dildohs': 'dildo', 'dramawhores': 'drama whores', 'edophiles': 'pedophiles', 'eurocommunist': 'communist', 'faggotkike': 'faggot', 'fantard': 'retard', 'fascismnazism': 'fascism', 'fascistisized': 'fascist', 'favremother': 'mother', 'fuxxxin': 'fucking', "g'damn": 'goddamn', 'harassmentat': 'harassment', 'harrasingme': 'harassing me', 'herfuc': 'motherfucker', 'hilterism': 'fascism', 'hitlerians': 'nazis', 'hitlerites': 'nazis', 'hubrises': 'pricks', 'idiotizing': 'idiotic', 'inadvandals': 'vandals', "jackass's": 'jackass', 'jiggabo': 'nigga', 'jizzballs': 'jizz balls', 'jmbass': 'dumbass', 'lejittament': 'legitimate', "m'igger": 'nigger', "m'iggers": 'niggers', 'motherfacking': 'motherfucker', 'motherfuckenkiwi': 'motherfucker', 'muthafuggas': 'niggas', 'nazisms': 'nazis', 'netsnipenigger': 'nigger', 'niggercock': 'nigger', 'niggerspic': 'nigger', 'nignog': 'nigga', 'niqqass': 'niggas', "non-nazi's": 'not a nazi', 'panamanians': 'mexicans', 'pedidiots': 'idiots', 'picohitlers': 'hitler', 'pidiots': 'idiots', 'poopia': 'poop', 'poopsies': 'poop', 'presumingly': 'obviously', 'propagandaanddisinformation': 'propaganda and disinformation', 'propagandaministerium': 'propaganda', 'puertoricans': 'mexicans', 'puertorricans': 'mexicans', 'pussiest': 'pussies', 'pussyitis': 'pussy', 'rayaridiculous': 'ridiculous', 'redfascists': 'fascists', 'retardzzzuuufff': 'retard', "revertin'im": 'reverting', 'scumstreona': 'scums', 'southamericans': 'mexicans', 'strasserism': 'fascism', 'stuptarded': 'retarded', "t'nonsense": 'nonsense', "threatt's": 'threat', 'titoists': 'communists', 'twatbags': 'douchebags', 'youbollocks': 'you bollocks'}
acronym_words = {"btw":"by the way", "yo": "you", "u": "you", "r": "are", "ur": "your", "ima": "i am going to", "imma": "i am going to", "i'ma":"i am going to", "cos":"because", "coz":"because", "stfu": "shut the fuck up", "wat": "what"}
CHARS_TO_REMOVE = '!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n“”’\'∞θ÷α•à−β∅³π‘₹´°£€\×™√²—'

english_stopwords = set(stopwords.words('english'))

#spell_checker = SpellChecker()

cont_patterns = [
    (r'(W|w)on\'t', r'will not'),
    (r'(C|c)an\'t', r'can not'),
    (r'(I|i)\'m', r'i am'),
    (r'(A|a)in\'t', r'is not'),
    (r'(\w+)\'ll', r'\g<1> will'),
    (r'(\w+)n\'t', r'\g<1> not'),
    (r'(\w+)\'ve', r'\g<1> have'),
    (r'(\w+)\'s', r'\g<1> is'),
    (r'(\w+)\'re', r'\g<1> are'),
    (r'(\w+)\'d', r'\g<1> would'),
]
patterns = [(re.compile(regex), repl) for (regex, repl) in cont_patterns]

#We will filter all characters except alphabet characters and some punctuation
valid_characters = " " + "@$" + "'!?-" + "abcdefghijklmnopqrstuvwxyz" + "abcdefghijklmnopqrstuvwxyz".upper()
valid_set = set(x for x in valid_characters)

## Funcs for processing

In [0]:
def replace_url(word):
    if "http://" in word or "www." in word or "https://" in word or "wikipedia.org" in word:
        return ""
    return word


def word_tokenize(sentence, tokenizer):
    sentence = sentence.replace("$", "s")
    sentence = sentence.replace("@", "a")    
    sentence = sentence.replace("!", " ! ")
    sentence = sentence.replace("?", " ? ")

    return tokenizer.tokenize(sentence)


def split_toxic_and_normal_parts(word, toxic_words):
    if word == "":
        return ""
    
    lower = word.lower()
    for toxic_word in toxic_words:
        start = lower.find(toxic_word)
        if start >= 0:
            end = start + len(toxic_word)
            result = " ".join([word[0:start], word[start:end], split_toxic_and_normal_parts(word[end:], toxic_words)])
            return result.replace("  ", " ").strip()
    return word


def normalize_by_dictionary(normalized_word, dictionary):
    result = []
    for word in normalized_word.split():
        if word == word.upper():
            if word.lower() in dictionary:
                result.append(dictionary[word.lower()].upper())
            else:
                result.append(word)
        else:
            if word.lower() in dictionary:
                result.append(dictionary[word.lower()])
            else:
                result.append(word)
    
    return " ".join(result)


def normalize_comment(comment, tokenizer, max_comment_length):
    comment = unidecode(comment)
    comment = comment[:max_comment_length]

    normalized_words = []
    
    # ('mother****ers', 'motherfuckers')
    # for w in astericks_words:
    #     if w[0] in comment:
    #         comment = comment.replace(w[0], w[1])
    #     if w[0].upper() in comment:
    #         comment = comment.replace(w[0].upper(), w[1].upper())
    
    for word in word_tokenize(comment, tokenizer):
        if word in english_stopwords: continue

        # # '(W|w)on\'t', r'will not'
        # for (pattern, repl) in patterns:
        #    word = re.sub(pattern, repl, word)

        if word == "." or word == ",":
            normalized_words.append(word)
            continue
        
        # drop url parts from links
        # word = replace_url(word)

        # replace single dots to whitespaces
        if word.count(".") == 1:
            word = word.replace(".", " ")

        # leave only legalized symbols
        filtered_word = "".join([x for x in word if x in valid_set])
                    
        #Kind of hack: for every word check if it has a toxic word as a part of it
        #If so, split this word by swear and non-swear part.
        #normalized_word = split_toxic_and_normal_parts(filtered_word, toxic_words)
        normalized_word = filtered_word

#         normalized_word = normalize_by_dictionary(normalized_word, hyphens_dict)
#         normalized_word = normalize_by_dictionary(normalized_word, merged_dict)
        
        # check misspellings
        # temp = []
        # for word in normalized_word.split():
        #   temp.append(spell_checker.correction(word))
        # normalized_word = " ".join(temp)
          
        # normalized_word = normalize_by_dictionary(normalized_word, fasttext_misspelings)
        normalized_word = normalize_by_dictionary(normalized_word, acronym_words)

        normalized_words.append(normalized_word)
        
    normalized_comment = " ".join(normalized_words)
    
    result = []
    for word in normalized_comment.split():
        # if word is upper
        if word.upper() == word:
            result.append(word)
        else:
            result.append(word.lower())
    
    #apparently, people on wikipedia love to talk about sockpuppets :-)
    result = " ".join(result)
    if "sock puppet" in result:
        result = result.replace("sock puppet", "sockpuppet")
    
    if "SOCK PUPPET" in result:
        result = result.replace("SOCK PUPPET", "SOCKPUPPET")
    
    return result


def preprocess_text(df, preprocessing_func, tokenizer, max_comment_length):
  text_ndarray = df.fillna('_na').values
  np_preprocessing_func = np.vectorize(preprocessing_func)

  preprocessed_text = np_preprocessing_func(text_ndarray, tokenizer, max_comment_length)

  print('Gained shape:', preprocessed_text.shape)
  return preprocessed_text


def tokenize_and_pad(text, tokenizer):

  tokenized_text = tokenizer.texts_to_sequences(text)
  padded_text = pad_sequences(tokenized_text, maxlen=MAX_LEN, dtype='int', value=0)

  print('Gained shape:', padded_text.shape)
  return padded_text

## Loading frames and processing them

In [0]:
TEXT_COLUMN = 'comment_text'
TARGET_COLS = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]

MAX_LEN = 220
max_tokens = 20000
max_comment_length = 20000 #We are going to truncate a comment if its length > threshold

In [0]:
%%time

train_df = pd.read_csv(train_filepath)
train_labels = train_df[TARGET_COLS].values

CPU times: user 733 ms, sys: 114 ms, total: 847 ms
Wall time: 1.12 s


In [0]:
%%time

tknzr = TweetTokenizer(strip_handles=False, reduce_len=True)

preprocessed_train = preprocess_text(train_df[TEXT_COLUMN], 
                                     normalize_comment, 
                                     tknzr,
                                     max_comment_length)

print(preprocessed_train[:3])

del train_df
gc.collect()

Gained shape: (159571,)
["explanation why edits made username hardcore metallica fan reverted ? they vandalisms , closure gas I voted new york dolls FAC . and please remove template talk page since i'm retired . ."
 "d'aww ! he matches background colour i'm seemingly stuck . thanks . talk , january , UTC"
 "hey man , i'm really trying edit war . it's guy constantly removing relevant information talking edits instead talk page . he seems care formatting actual info ."]
CPU times: user 59.8 s, sys: 3.02 s, total: 1min 2s
Wall time: 1min 2s


In [0]:
%%time

test_df = pd.read_csv(test_filepath)

preprocessed_test = preprocess_text(test_df[TEXT_COLUMN], 
                                    normalize_comment, 
                                    tknzr,
                                    max_comment_length)

del test_df, tknzr
gc.collect()
# y_train = np.apply_along_axis(document_vector, 1, tokenized_test)

Gained shape: (153164,)
CPU times: user 54.5 s, sys: 2.87 s, total: 57.4 s
Wall time: 57.7 s


# Training model

## Preparing features

In [0]:
# word_index = tokenizer.word_index
# print('Found %s unique tokens.' %len(word_index))


# # %%time
# # embedding_matrix = np.load('../input/embedding-2/embedding_matrix_big.npy')


# embedding_len = 25
# #g = GloveEmbedding('twitter', d_emb=embedding_len, show_progress=True)
# words = pd.read_csv(os.path.join(HW_folder, "glove.twitter.27B.{}d.txt".format(embedding_len)), 
#                     sep=" ", index_col=0, header=None, quoting=csv.QUOTE_NONE)


# vocab_words = words[words.index.isin(list(word_index.keys()))]
# print('vocab_words.shape', vocab_words.shape)


# def get_vec(word, words_df):
#   return words_df.loc[word].values


# # embedding_matrix = np.zeros((len(vocab), embedding_len))
# # for word, emb in tqdm(embeddings_dict.items(), total=len(embeddings_dict), mininterval=10):
# #     index = list(embeddings_dict.keys()).index(word)
# #     if emb is not None:
# #       embedding_matrix[index] = emb


# num_words = min(max_tokens, len(word_index) + 1)
# embedding_matrix = np.zeros((num_words, embedding_len)) # zeroth row for UNK
# for word, i in word_index.items():
#   if i >= max_tokens:
#     continue
#   try:
#     embedding_vector = get_vec(word, vocab_words)
#   except:
#     continue
#   if embedding_vector is not None:
#     # words not found in embedding index will be all-zeros.
#     embedding_matrix[i] = embedding_vector


# print('embedding_matrix.shape', embedding_matrix.shape)

In [0]:
%%time

tfidf_vec = TfidfVectorizer(tokenizer=None,
                            min_df=2,
                            max_df=.625,
                            strip_accents='unicode',
                            use_idf=1,
                            smooth_idf=True,
                            sublinear_tf=True,
                            max_features=max_tokens)#, stop_words=stop_words_en)

train_features = tfidf_vec.fit_transform(preprocessed_train)
test_features = tfidf_vec.transform(preprocessed_test)

del preprocessed_train, preprocessed_test, tfidf_vec
gc.collect()

CPU times: user 20.3 s, sys: 508 ms, total: 20.8 s
Wall time: 20.9 s


## Build and fit

### Log Reg

In [0]:
submission = pd.read_csv(submission_path)

In [0]:
# %%time

# def document_vector(doc, embedding_matrix=embedding_matrix):
#     """Create document vectors by averaging word vectors. Remove out-of-vocabulary words."""
#     doc = [embedding_matrix[index] for index in doc]
#     return np.mean(doc, axis=0)


# x_train = np.apply_along_axis(document_vector, 1, tokenized_train)

In [0]:
models = {
    'RandomForestClassifier': RandomForestClassifier(),
    'LogisticRegression': LogisticRegression()
}

# the optimisation parameters for each of the above models
params = {
    'RandomForestClassifier':{ 
        "n_estimators": [100],
        "max_features": ["auto"],
        "bootstrap": [True],
        "criterion": ['gini', 'entropy'],
        "oob_score": [True, False]
            },
    'LogisticRegression': {
        'solver': ['newton-cg', 'sag', 'lbfgs'],
        'C': [0.01, 0.1, 0.5, 1, 10, 100, 1000]
        }  
}

In [0]:
%%time

for i, class_name in enumerate(TARGET_COLS):
    print('**Processing {} comments...**'.format(class_name))

    best_score = 0
    best_model_with_params = tuple()

    for name in models.keys():
      print(' **Processing {} model...**'.format(name))
      est = models[name]
      est_params = params[name]
      gscv = GridSearchCV(estimator=est, param_grid=est_params, iid=False, cv=4, 
                          verbose=1, n_jobs=-1, scoring='roc_auc')
      gscv.fit(train_features, train_labels[:, i])
      print("Best score for the model\n{}\nis:{}".format(gscv.best_estimator_, gscv.best_score_))

      if gscv.best_score_ > best_score:
        best_score = gscv.best_score_
        best_model = gscv.best_estimator_ 
    
    best_model.fit(train_features, train_labels[:, i])
    submission[class_name] = best_model.predict_proba(test_features)[:, 1]

    print("\nBest score for the class {} is {}\n".format(class_name, best_score))

**Processing toxic comments...**
 **Processing RandomForestClassifier model...**
Fitting 4 folds for each of 4 candidates, totalling 16 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.


In [0]:
submission.head()

## Cheaty evaluating on test labels

In [0]:
# labels for the test data; value of -1 indicates it was not used for scoring; (Note: file added after competition close!)
test_labels_df = pd.read_csv(test_labels_filepath)

In [0]:
test_labels_df = test_labels_df[(test_labels_df["toxic"] != -1) &
                                (test_labels_df["severe_toxic"] != -1) &
                                (test_labels_df["obscene"] != -1) &
                                (test_labels_df["threat"] != -1) &
                                (test_labels_df["insult"] != -1) &
                                (test_labels_df["identity_hate"] != -1)]
test_labels_df.shape                               

In [0]:
submission_to_evaluate = submission[submission['id'].isin(test_labels_df['id'].values)]
submission_to_evaluate.shape

In [0]:
scores = []
for class_name in TARGET_COLS:
    train_target = test_labels_df[class_name]
    train_predicted = submission_to_evaluate[class_name]

    cv_score = np.mean(roc_auc_score(train_target.values, train_predicted.values))
    scores.append(cv_score)
    print('CV score for class {} is {}'.format(class_name, cv_score))

print('Total CV score is {}'.format(np.mean(scores)))

In [0]:
submission.to_csv("submission_tfidf_sklean.csv", index = False)

TFIDF_+_logreg  0.97620